# Titanic DataSet Kaggle Challenge

[1] Crisp DM Vorgehensmodel https://www.bigdata-insider.de/was-ist-crisp-dm-a-815478/

## Step 1: Geschäftsverständnis

(engl. Business Understanding) In der Phase des Geschäftsverständnisses geht es darum, die konkreten Ziele und Anforderungen für das Data Mining festzulegen. Ergebnis dieser Phase ist die Formulierung der Aufgabenstellung und die Beschreibung der geplanten groben Vorgehensweise. [1]

<img src="visuals/titanic_sketch.png">

Beschreibung der Titanic Story hier einfügen

Quelle: https://de.wikipedia.org/wiki/RMS_Titanic

Todo: Timeline aus Kaggle kopieren

Definition der Aufgabe: "Vorhersage" ob eine Passagier_In den Untergang der Titanic überlebt hat

## Step 2: Datenverständnis 

(engl. Data Understanding) Im Rahmen des Datenverständnisses wird versucht, sich einen ersten Überblick über die zur Verfügung stehenden Daten und deren Qualität zu verschaffen. Es erfolgt eine Analyse und Bewertung der Datenqualität. Probleme mit der Qualität der vorhandenen Daten in Bezug auf die in der vorherigen Phase festgelegten Aufgabenstellung sind zu benennen. [1]

### 2.1 Vorbereitung der Entwicklungsumgebung
Import der notwendigen Python Bibliotheken

In [ ]:
import pandas as pd             # Pandas Bibliothek für Datenverarbeitung, siehe https://pandas.pydata.org/
import matplotlib.pyplot as plt # Matplotlib für die Datenvisualisierung, siehe https://matplotlib.org/
import seaborn as sns           # Seaborn für die Datenvisualisierung, siehe https://seaborn.pydata.org/
import numpy as np              # Numpy für effiziente Rechenoperationen, siehe https://numpy.org/
import os                       # OS für den Zugriff auf externe Daten, siehe https://docs.python.org/3/library/os.html


# Sklearn ML-Algorithmen
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

# Sklearn Modellierungs-Helfer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, normalize, PowerTransformer
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.tree import export_graphviz

Formatierung der Mark-up Tabellen: linksbündige Ausrichtung

In [ ]:
%%html
<style>
table {float:left}
</style>

In [ ]:
%matplotlib inline

### 2.2 Datenimport
Import und merge der CSV Daten <br>
Quelle: https://www.kaggle.com/c/titanic/data

In [ ]:
df_train = pd.read_csv("data/train.csv")                      # Trainingsdaten-Datei einlesen
df_test  = pd.read_csv("data/test.csv")                       # Testdaten-Datei einlesen
df = pd.concat([df_train, df_test], ignore_index = True)      # beide Datensätze mergen, Trainingsdatensatz 0:892, Testdatensatz 892:1310

### 2.3 Datenerkundung
Einen Überblick über die vorhandenen Daten verschaffen <br>
Quellen: https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy

#### Beschreibung der Daten

| Variable | Bedeutung | Beschreibung |
| :- | :- | :- |
| PassengerId | Passagier_In-Nummer | Zahl zwischen 1 und 1309 |
| Survived | Hat die Passagier_In den Untergang überlebt | 0 = Nein, 1 = Ja |
| Pclass | Ticket-Klasse | 1 = 1. Klasse, 2 = 2. Klasse, 3 = 3. Klasse
| Name | Name der Passagier_In | Nachname, Vorname |
| Sex | Geschlecht der Passagier_In | male = Mann, female = Frau |
| Age | Alter der Passagier_In |
| SibSp | Anzahl der Geschwister/ Ehepartner an Board |
| Parch | Anzahl der Kinder/ Eltern an Board |
| Ticket | Ticket-Nummer |
| Fare | Ticket-Preis |
| Cabin | Kabinennummer |
| Embarked | Einschiffungshafen | C = Cherbourg, Q = Queenstown, S = Southampton |

In [ ]:
df.head(10) # Tabellarische Darstellung der ersten 10 Datensätze erzeugen

In [ ]:
df.sample(10) # Tabellarische Darstellung von 10 zufälligen Datensätze erzeugen

In [ ]:
df.info() # Beschreibung des gesamten Datensatzes: Fehlende Werte und Datentype

In [ ]:
df.describe(include = "all") # Statistische Beschreibung des Datensatzes

### 2.4 Visualisierung

In [ ]:
# Alter der Pasagier_Innen
sns.distplot(a = df["Age"], bins = 16, kde = False)

In [ ]:
# Alter der Pasagier_Innen
sns.distplot(a = df["Fare"], bins = 20, kde = False)

In [ ]:
# Alter der Pasagier_Innen vs Überleben
sns.catplot(x = "Sex", y = "Survived", kind = "bar", data = df)

In [ ]:
# Verteilung der Klassen
sns.catplot(x = "Pclass", y = "Survived", kind = "bar", data = df)

In [ ]:
# Alter vs Überleben
sns.histplot(x = "Age", hue = "Survived", data = df)

In [ ]:
# Klasse vs Überleben
sns.displot(x = "Pclass", hue = "Survived", data = df, discrete = True)

### 2.5 Erkenntnisse

#### Beschreibung der Daten

| Variable | Bedeutung | Beschreibung |Formatierung fehlerhaft | Formatierung soll | Fehlende Werte |
| :--- | :--- | :--- | :--- | :--- | :--- |
| PassengerId | Passagier_In-Nummer | Zahl zwischen 1 und 1309 | ja | Integer | nein |
| Survived | Hat die Passagier_In den Untergang überlebt | 0 = Nein, 1 = Ja | nein | | nein |
| Pclass | Ticket-Klasse | 1 = 1. Klasse, 2 = 2. Klasse, 3 = 3. Klasse ja | Integer | | ja |
| Name | Name der Passagier_In | Nachname, Vorname | nein | | nein | 
| Sex | Geschlecht der Passagier_In | male = Mann, female = Frau | nein | | ja |
| Age | Alter der Passagier_In | | ja | Integer | ja |
| SibSp | Anzahl der Geschwister/ Ehepartner an Board | | nein | | nein |
| Parch | Anzahl der Kinder/ Eltern an Board | | nein | | nein |
| Ticket | Ticket-Nummer | |  nein | | nein |
| Fare | Ticket-Preis | |  nein | | ja |
| Cabin | Kabinennummer | |  nein | | ja |
| Embarked | Einschiffungshafen | C = Cherbourg, Q = Queenstown, S = Southampton |  nein | | ja |

## Step 3: Datenvorbereitung
(engl. Data Preparation) Die Datenvorbereitung dient dazu, einen finalen Datensatz zu erstellen, der die Basis für die nächste Phase der Modellierung bildet. [1]

### 2.1 Auffüllen der fehlenden Werte 

In [ ]:
df["Age"]      = df['Age'].fillna(df['Age'].median())            # Alter mit dem Median auffüllen
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0]) # Einschiffhafen mit dem Modus auffüllen
df['Fare']     = df['Fare'].fillna(df['Fare'].median())          # Ticket-Preis mit dem Median auffüllen

### 2.2 Umwandlung der Eingangsdaten

In [ ]:
df["PassengerId"] = df["PassengerId"].astype("int")              # Passagier_In-Nummer in Integer umwandeln
#df["Survived"]    = df["Survived"].astype("boolean")             # Überleben in Boolean umwandeln
df["Age"]         = df["Age"].astype("int")                      # Alter in Integer umwandeln

In [ ]:
# Überprüfung der Umwandlung
df.info()

### 2.3 Zusätzliche Features anlegen

In [ ]:
# Familiengröße
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# Einzelreisende
df['IsAlone']    = 1
df['IsAlone'].loc[df['FamilySize'] > 1] = 0

# Ticketkathegorie
df['FareBin']    = pd.qcut(df['Fare'], 4)

# Altersgruppe
df['AgeBin']     = pd.cut(df['Age'].astype(int), 5)

# Kabineninhaber_In
df['CabinOwn']   = 1
df['CabinOwn']   = df['CabinOwn'] - df['Cabin'].isna().astype("int")

### 2.4 Überflüssige Features löschen

In [ ]:
# Löschen der Spalten Name, Cabin und Ticket
df = df.drop(["Name", "Cabin", "Ticket"],axis = 1)

### 2.5 Encoding der Features

In [ ]:
label               = LabelEncoder()                      # Encoder Objekt anlegen
df['Sex_Code']      = label.fit_transform(df['Sex'])      # Encodiertes Geschlecht
df['Embarked_Code'] = label.fit_transform(df['Embarked']) # Encodierter Einschiffhafen
df['AgeBin_Code']   = label.fit_transform(df['AgeBin'])   # Encodiertes Alter
df['FareBin_Code']  = label.fit_transform(df['FareBin'])  # Encodierter Ticketpreis

### 2.6 Normalisierung und Standardisierung

In [ ]:
df['Age_normed'] = (df['Age']-df['Age'].mean())/df['Age'].std()

### 2.7 Dummy Variablen bilden 

In [ ]:
#df_dummy = pd.get_dummies(df["Sex", "Pclass", "FamilySize", "Embarked"]) 
#df_dummy_sex      = pd.get_dummies(df["Sex"], prefix = "sex")
#df_dummy_class    = pd.get_dummies(df["Pclass"], prefix = "Pclass")
#df_dummy_embarked = pd.get_dummies(df["Embarked"], prefix = "Embarked")

In [ ]:
#df = pd.concat([df, df_dummy_sex, df_dummy_class, df_dummy_embarked], axis = 1)

In [ ]:
#df.info()

### 2.6 Aufteilung der Datensätze

In [ ]:
df_train = df[0:891]
df_train.info()

In [ ]:
df_test  = df[891:]
df_test.info()

### 2.7 Datenexploration durch Visualisierung

In [ ]:
# Alter der Pasagier_Innen
sns.histplot(x = "Age", hue = "Sex", data = df_train)

In [ ]:
sns.catplot(x = "Sex", y = "Survived", kind = "bar", data = df_train)

In [ ]:
sns.catplot(x = "Pclass", y = "Survived", kind = "bar", data = df_train)

In [ ]:
sns.catplot(x = "AgeBin", y = "Survived", kind = "bar", data = df_train)

In [ ]:
# Fare vs Survive
sns.catplot(x = "FareBin", y = "Survived", kind = "bar", data = df_train)

In [ ]:
#pair plots of entire dataset
#pp = sns.pairplot(df_train, hue = 'Survived', palette = 'deep', size=1.2, diag_kind = 'kde', diag_kws=dict(shade=True), plot_kws=dict(s=10) )
#pp.set(xticklabels=[])

### 2.9 Modellierung
(engl. Modeling) <br> 
Im Rahmen der Modellierung werden die für die Aufgabenstellung geeigneten Methoden des Data Minings auf den in der Datenvorbereitung erstellten Datensatz angewandt. <br> 
Typisch für diese Phase sind die Optimierung der Parameter und die Erstellung mehrerer Modelle.

#### 2.9.1 Train - Test - Split

In [ ]:
params = ["Fare"]
train1_x, test1_x, train1_y, test1_y = model_selection.train_test_split(df_train[params], df_train["Survived"], random_state = 1)

In [ ]:
train1_x.head(5)

In [ ]:
train1_y.head(5)

#### 2.9.2 Training eines Logistischen Regressions Models

In [ ]:
# Model initialisieren und trainieren
log_reg = linear_model.LogisticRegression()
log_reg = log_reg.fit(train1_x, train1_y)

In [ ]:
# Model für Prognose verwenden und Prognose ausgeben
log_reg_prediction = log_reg.predict(test1_x)
print("Prognostizierte Werte")
print(log_reg_prediction)

In [ ]:
# Wahre Werte ausgeben
test1_y_np = test1_y.to_numpy() # Pandas Serie in Numpy Array umwandeln
print("Wahre Werte")
print(test1_y_np)

In [ ]:
# Vergleich zwischen Prognose und wahren Werten ausgeben
difference_pred_true = log_reg_prediction - test1_y_np
print("Vergleich Prognose vs. Wahre Werte")
print(difference_pred_true)

In [ ]:
# Berechnen der Vorhersagegenauigkeit
count_zeros  = np.count_nonzero(log_reg_prediction-test1_y.to_numpy() == 0. , axis=0) # Nullen im Vergleichs Array zählen
count_values = len(log_reg_prediction-test1_y.to_numpy())                             # Anzahl der Werte im Vergleichs Array zählen
score        = count_zeros / count_values                                             # Anteil der korrekt vorhergesagten Werte zählen
print("Genauigkeit des Models: %1.3f %%"%(score))

In [ ]:
# Berechnung der Genauigkeit mit einer Zeile
score = log_reg.score(test1_x, test1_y)
print("Genauigkeit des Models: %1.3f %%"%(score))

In [ ]:
# Funktion für die Erstellung einer ansehnlichen Confusion Matrix
# Credit: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Confusion Matrix erstellen
conf_matrix = metrics.confusion_matrix(log_reg_prediction, test1_y.to_numpy())
classes = ["verstorben", "überlebt"]
plot_confusion_matrix(conf_matrix, classes, normalize = True)

#### 2.9.2 Training eines Logistischen Regressions Models mit mehreren Inputs

In [ ]:
params = ["Fare", "Age", "FamilySize"]
train2_x, test2_x, train2_y, test2_y = model_selection.train_test_split(df_train[params], df_train["Survived"], random_state = 1)

In [ ]:
# Model initialisieren und trainieren
log_reg = linear_model.LogisticRegression()
log_reg = log_reg.fit(train2_x, train2_y)

In [ ]:
# Model für Prognose verwenden und Prognose ausgeben
log_reg_prediction = log_reg.predict(test2_x)

In [ ]:
# Berechnung der Genauigkeit mit einer Zeile
score = log_reg.score(test2_x, test2_y)
print("Genauigkeit des Models: %1.3f %%"%(score))

In [ ]:
# Confusion Matrix erstellen
conf_matrix = metrics.confusion_matrix(log_reg_prediction, test2_y.to_numpy())
classes = ["verstorben", "überlebt"]
plot_confusion_matrix(conf_matrix, classes, normalize = True)

#### 2.9.3 Vergleich mit dem Zufall (Münzwurf)

In [ ]:
# Zufälliges Vorhersage erstellen
random_prediction = np.random.randint(0,2,size = len(test2_y)).astype(float)

In [ ]:
print(random_prediction)

In [ ]:
# Wahre Werte ausgeben
test2_y_np = test2_y.to_numpy() # Pandas Serie in Numpy Array umwandeln

# Vergleich zwischen Zufall und wahren Werten ausgeben
difference_pred_true = random_prediction - test2_y_np

# Berechnen der Vorhersagegenauigkeit
count_zeros  = np.count_nonzero(random_prediction - test2_y.to_numpy() == 0. , axis=0) # Nullen im Vergleichs Array zählen
count_values = len(random_prediction - test2_y.to_numpy())                             # Anzahl der Werte im Vergleichs Array zählen
score        = count_zeros / count_values                                              # Anteil der korrekt vorhergesagten Werte zählen
print("Genauigkeit des Models: %1.3f %%"%(score))

# Confusion Matrix erstellen
conf_matrix = metrics.confusion_matrix(random_prediction, test2_y.to_numpy())
classes = ["verstorben", "überlebt"]
plot_confusion_matrix(conf_matrix, classes, normalize = True)

#### 2.9.4 Training eines Support Vector Machine Models

In [ ]:
# Parameterdefinition
params = ["CabinOwn", "Sex_Code", "Embarked_Code", "AgeBin_Code", "FareBin_Code"]

# Train-Test-Split
train4_x, test4_x, train4_y, test4_y = model_selection.train_test_split(df_train[params], df_train["Survived"], random_state = 1)

In [ ]:
# Model initialisieren, trainieren und Vorhersage erstellen
svm = svm.SVC(decision_function_shape='ovo')
svm = svm.fit(train4_x, train4_y)
svm_prediction = svm.predict(test4_x)

In [ ]:
# Model score
score = svm.score(test4_x, test4_y)
print("Genauigkeit des Models: %1.3f %%"%(score))

In [ ]:
# Confusion Matrix erstellen
conf_matrix = metrics.confusion_matrix(svm_prediction, test4_y.to_numpy())
classes = ["verstorben", "überlebt"]
plot_confusion_matrix(conf_matrix, classes, normalize = True)

#### 2.9.5 Training eines k-Next-Neighbors Models

In [ ]:
# Parameterdefinition
params = ["CabinOwn", "Sex_Code", "Embarked_Code", "AgeBin_Code", "FareBin_Code"]

# Train-Test-Split
train5_x, test5_x, train5_y, test5_y = model_selection.train_test_split(df_train[params], df_train["Survived"], random_state = 1)

In [ ]:
# Model initialisieren, trainieren und Vorhersage erstellen
knn = neighbors.KNeighborsClassifier()
knn = knn.fit(train5_x, train5_y)
knn_prediction = knn.predict(test5_x)

In [ ]:
# Model score
score = knn.score(test5_x, test5_y)
print("Genauigkeit des Models: %1.3f %%"%(score))

In [ ]:
# Confusion Matrix erstellen
conf_matrix = metrics.confusion_matrix(knn_prediction, test5_y.to_numpy())
classes = ["verstorben", "überlebt"]
plot_confusion_matrix(conf_matrix, classes, normalize = True)

#### 2.9.6 Training eines Random Forest Classifiers

In [ ]:
# Parameterdefinition
params = ["CabinOwn", "Sex_Code", "Embarked_Code", "AgeBin_Code", "FareBin_Code"]

# Train-Test-Split
train6_x, test6_x, train6_y, test6_y = model_selection.train_test_split(df_train[params], df_train["Survived"], random_state = 1)

In [ ]:
# Model initialisieren, trainieren und Vorhersage erstellen
rfc = ensemble.RandomForestClassifier(n_estimators = 20, max_depth = 4)
rfc = rfc.fit(train6_x, train6_y)
rfc_predicition = rfc.predict(test6_x)

In [ ]:
# Model score
score = rfc.score(test6_x, test6_y)
print("Genauigkeit des Models: %1.3f %%"%(score))

In [ ]:
# Confusion Matrix erstellen
conf_matrix = metrics.confusion_matrix(rfc_predicition, test6_y.to_numpy())
classes = ["verstorben", "überlebt"]
plot_confusion_matrix(conf_matrix, classes, normalize = True)

#### 2.9.7 Kombination der Vorhersagen

In [ ]:
features = pd.DataFrame(data = {"log_reg" : log_reg.predict(train2_x),
                                "svm" : svm.predict(train4_x), 
                                "kNN" : knn.predict(train5_x),
                                "rfc" : rfc.predict(train6_x), 
                                "true" : train1_y.to_numpy()})

In [ ]:
feature_list = ["log_reg", "kNN", "svm", "rfc", "true"]
for feature in feature_list:
    features[feature] = features[feature].astype(bool)
features


In [ ]:
# Parameterdefinition
params = ["log_reg", "kNN", "svm", "rfc"]

# Train-Test-Split
train0_x, test0_x, train0_y, test0_y = model_selection.train_test_split(features[params], features["true"], random_state = 1)

In [ ]:
# Model initialisieren, trainieren und Vorhersage erstellen
ensemble_model = ensemble.RandomForestClassifier(n_estimators = 3, max_depth = 3, criterion = "entropy")
ensemble_model= ensemble_model.fit(train0_x, train0_y)
ensemble_model_prediction = ensemble_model.predict(test0_x)

In [ ]:
# Model score
score = ensemble_model.score(test0_x, test0_y)
print("Genauigkeit des Models: %1.3f %%"%(score))

In [ ]:
# Confusion Matrix erstellen
conf_matrix = metrics.confusion_matrix(ensemble_model_prediction, test0_y.to_numpy())
classes = ["verstorben", "überlebt"]
plot_confusion_matrix(conf_matrix, classes, normalize = True)

In [ ]:
# Extract single tree
estimator = ensemble_model.estimators_[0]

# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = params,
                class_names = ["verstorben", "überlebt"],
                rounded = True, proportion = False, 
                precision = 1, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')


In [ ]:
# Extract single tree
estimator = ensemble_model.estimators_[1]

# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = params,
                class_names = ["verstorben", "überlebt"],
                rounded = True, proportion = False, 
                precision = 1, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

#### 3.1 Vorhersage auf ungesehene Daten

In [ ]:
# Daten laden
df_test.head()

In [ ]:
params_continous = ["Fare", "Age", "FamilySize"]
params_classes = ["CabinOwn", "Sex_Code", "Embarked_Code", "AgeBin_Code", "FareBin_Code"]

In [ ]:
df_test_preds = pd.DataFrame(log_reg.predict(df_test[params_continous]))

In [ ]:
df_test_preds = pd.DataFrame(data = {"log_reg" : log_reg.predict(df_test[params_continous]).astype(int),
                                     "svm"     : svm.predict(df_test[params_classes]).astype(int), 
                                     "kNN"     : knn.predict(df_test[params_classes]).astype(int),
                                     "rfc"     : rfc.predict(df_test[params_classes]).astype(int)})
df_test_preds.head()

In [ ]:
df_test_preds["ensemble"] = ensemble_model.predict(df_test_preds).astype(int)
df_test_preds

In [ ]:
export = pd.DataFrame(data = {"PassengerId" : df_test["PassengerId"].values, 
                              "Survived" : df_test_preds["ensemble"].astype(int)})

In [ ]:
export.to_csv("submission.csv", index = False)